In [1]:
import pandas as pd
import numpy as np
from utils.utils import *

In [2]:
prev = pd.read_csv(DIR+'previous_application.csv.zip')

In [3]:
prev, cat_cols = one_hot_encoder(prev)
# Days 365.243 values -> nan
prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)
# Add feature: value ask / value received percentage
prev['APP_CREDIT_PERC'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']
# Previous applications numeric features
num_aggregations = {
    'AMT_ANNUITY': [ 'max', 'mean'],
    'AMT_APPLICATION': [ 'mean'],
    'AMT_CREDIT': [ 'max', 'mean'],
    'APP_CREDIT_PERC': [ 'max', 'mean'],
    'AMT_DOWN_PAYMENT': [ 'max', 'mean'],
    'AMT_GOODS_PRICE': [ 'max', 'mean'],
    'HOUR_APPR_PROCESS_START': [ 'max', 'mean'],
    'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
    'DAYS_DECISION': ['min', 'max', 'mean'],
    'CNT_PAYMENT': ['mean', 'sum'],
}
# Previous applications categorical features
cat_aggregations = {}
for cat in cat_cols:
    cat_aggregations[cat] = ['mean']

prev_agg = prev.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])
prev_agg_col = prev_agg.columns
# Previous Applications: Approved Applications - only numerical features
approved = prev[prev['NAME_CONTRACT_STATUS_Approved'] == 1]
approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
approved_agg.columns = pd.Index(['APPROVED_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
prev_agg = prev_agg.join(approved_agg, how='left', on='SK_ID_CURR')
# Previous Applications: Refused Applications - only numerical features
refused = prev[prev['NAME_CONTRACT_STATUS_Refused'] == 1]
refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
refused_agg.columns = pd.Index(['REFUSED_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
prev_agg = prev_agg.join(refused_agg, how='left', on='SK_ID_CURR')

# add
#for e in prev_agg_col.tolist():
#    prev_agg['ARRATE_'+ e[5:]] = prev_agg['APPROVED_' + e[5:]] / prev_agg['REFUSED_' + e[5:]]

del refused, refused_agg, approved, approved_agg, prev
gc.collect()

106

In [4]:
np.set_printoptions(threshold=np.inf)

In [5]:
np.array(prev_agg.columns)

array(['PREV_AMT_ANNUITY_MAX', 'PREV_AMT_ANNUITY_MEAN',
       'PREV_AMT_APPLICATION_MEAN', 'PREV_AMT_CREDIT_MAX',
       'PREV_AMT_CREDIT_MEAN', 'PREV_APP_CREDIT_PERC_MAX',
       'PREV_APP_CREDIT_PERC_MEAN', 'PREV_AMT_DOWN_PAYMENT_MAX',
       'PREV_AMT_DOWN_PAYMENT_MEAN', 'PREV_AMT_GOODS_PRICE_MAX',
       'PREV_AMT_GOODS_PRICE_MEAN', 'PREV_HOUR_APPR_PROCESS_START_MAX',
       'PREV_HOUR_APPR_PROCESS_START_MEAN', 'PREV_RATE_DOWN_PAYMENT_MIN',
       'PREV_RATE_DOWN_PAYMENT_MAX', 'PREV_RATE_DOWN_PAYMENT_MEAN',
       'PREV_DAYS_DECISION_MIN', 'PREV_DAYS_DECISION_MAX',
       'PREV_DAYS_DECISION_MEAN', 'PREV_CNT_PAYMENT_MEAN',
       'PREV_CNT_PAYMENT_SUM', 'PREV_NAME_CONTRACT_TYPE_Cash loans_MEAN',
       'PREV_NAME_CONTRACT_TYPE_Consumer loans_MEAN',
       'PREV_NAME_CONTRACT_TYPE_Revolving loans_MEAN',
       'PREV_NAME_CONTRACT_TYPE_XNA_MEAN',
       'PREV_NAME_CONTRACT_TYPE_nan_MEAN',
       'PREV_WEEKDAY_APPR_PROCESS_START_FRIDAY_MEAN',
       'PREV_WEEKDAY_APPR_PROCESS_START_M

In [6]:
prev_agg = prev_agg.reset_index()

In [7]:
prev_agg.to_pickle('./edit/fixed_data/prev.pkl')